# **Problem Statement**

## Business Context

A sales forecast is a prediction of future sales revenue based on historical data, industry trends, and the status of the current sales pipeline. Businesses use the sales forecast to estimate weekly, monthly, quarterly, and annual sales totals. A company needs to make an accurate sales forecast as it adds value across an organization and helps the different verticals to chalk out their future course of action.

Forecasting helps an organization plan its sales operations by region and provides valuable insights to the supply chain team regarding the procurement of goods and materials. An accurate sales forecast process has many benefits which include improved decision-making about the future and reduction of sales pipeline and forecast risks. Moreover, it helps to reduce the time spent in planning territory coverage and establish benchmarks that can be used to assess trends in the future.

## Objective

SuperKart is a retail chain operating supermarkets and food marts across various tier cities, offering a wide range of products. To optimize its inventory management and make informed decisions around regional sales strategies, SuperKart wants to accurately forecast the sales revenue of its outlets for the upcoming quarter.

To operationalize these insights at scale, the company has partnered with a data science firm—not just to build a predictive model based on historical sales data, but to develop and deploy a robust forecasting solution that can be integrated into SuperKart’s decision-making systems and used across its network of stores.

## Data Description

The data contains the different attributes of the various products and stores.The detailed data dictionary is given below.

- **Product_Id** - unique identifier of each product, each identifier having two letters at the beginning followed by a number.
- **Product_Weight** - weight of each product
- **Product_Sugar_Content** - sugar content of each product like low sugar, regular and no sugar
- **Product_Allocated_Area** - ratio of the allocated display area of each product to the total display area of all the products in a store
- **Product_Type** - broad category for each product like meat, snack foods, hard drinks, dairy, canned, soft drinks, health and hygiene, baking goods, bread, breakfast, frozen foods, fruits and vegetables, household, seafood, starchy foods, others
- **Product_MRP** - maximum retail price of each product
- **Store_Id** - unique identifier of each store
- **Store_Establishment_Year** - year in which the store was established
- **Store_Size** - size of the store depending on sq. feet like high, medium and low
- **Store_Location_City_Type** - type of city in which the store is located like Tier 1, Tier 2 and Tier 3. Tier 1 consists of cities where the standard of living is comparatively higher than its Tier 2 and Tier 3 counterparts.
- **Store_Type** - type of store depending on the products that are being sold there like Departmental Store, Supermarket Type 1, Supermarket Type 2 and Food Mart
- **Product_Store_Sales_Total** - total revenue generated by the sale of that particular product in that particular store


# **Installing and Importing the necessary libraries**

In [ ]:
#Installing the libraries with the specified versions
!pip install numpy==2.0.2 pandas==2.2.2 scikit-learn==1.6.1 matplotlib==3.10.0 seaborn==0.13.2 joblib==1.4.2 xgboost==2.1.4 requests==2.32.3 huggingface_hub==0.30.1 -q

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# For splitting the dataset
from sklearn.model_selection import train_test_split

# Libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 100)


# Libraries different ensemble classifiers
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor,
)
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

# Libraries to get different metric scores
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)

# To create the pipeline
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# To tune different models and standardize
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# To serialize the model
import joblib

# os related functionalities
import os

# API request
import requests

# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi
import tensorflow as tf

In [ ]:
# Setting the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
tf.keras.utils.set_random_seed(812)

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()

# **Loading the dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading the data
superkartdata = pd.read_csv('/content/drive/MyDrive/SuperKart.csv')

In [ ]:
# Creating a copy of data
dataset = superkartdata.copy()

# **Data Overview**

In [ ]:
# Checking the shape of datset
dataset.shape

There are 8763 rows and 12 columns in the dataset.

In [ ]:
# Checking the variables details
dataset.info()

The dataset contains 7 categorical columns and 5 numerical columns.

In [ ]:
# Checking the statistical summary
dataset.describe().T

The dataset reflects moderate feature variability with highly dispersed sales outcomes, justifying the use of robust, non-linear regression model.

In [ ]:
# Printing the first five rows of dataset
dataset.head()

In [ ]:
# Printing the last five rows of dataset
dataset.tail()

In [ ]:
# Printing the 30 sample rows of dataset
dataset.sample(30)

In [ ]:
# Checking for null values
dataset.isnull().sum()

The dataset doesnot contains null values.

In [ ]:
# Checking for duplicate values
dataset.duplicated().sum()

The dataset doesnot contains duplicate values.

A typo has been identified in the Product_Sugar_Content column. While the data dictionary specifies three valid categories—low sugar, regular, and no sugar—the value regular is mistakenly stored as reg in several records.

### Correcting the typo in Product_Sugar_Content column

In [ ]:
# Clean and standardize the 'Product_Sugar_Content' column
dataset['Product_Sugar_Content'] = (
    dataset['Product_Sugar_Content']
    .str.strip()          # Remove leading and trailing whitespaces
    .str.lower()          # Convert all values to lowercase for consistency
    .replace({            # Standardize category names
        'reg': 'regular', # Replace short form with full word
        'low sugar': 'low sugar',
        'no sugar': 'no sugar'
    })
)



In [ ]:
# Checking the value counts for each category
dataset['Product_Sugar_Content'].value_counts()

So, the typo has been corrected now.

# **Exploratory Data Analysis (EDA)**

## Univariate Analysis

In [ ]:
# function to create histogram and box plots

def histogram_boxplot(data, feature, figsize=(15, 10), bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (15,10))
    kde: whether to show the density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="lightgreen"
    )  # boxplot will be created and a triangle will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=True, ax=ax_hist2, bins=bins, color="indigo"
    ) if bins else sns.histplot(
        data=data, x=feature, kde=True, ax=ax_hist2,color="indigo"
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="red", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="red", linestyle="-"
    )  # Add median to the histogram



In [ ]:
# function to create labeled barplots

def labeled_barplot(data, feature, perc=False, n=None):
    """
    Barplot with percentage at the top

    data: dataframe
    feature: dataframe column
    perc: whether to display percentages instead of count (default is False)
    n: displays the top n category levels (default is None, i.e., display all levels)
    """

    total = len(data[feature])  # length of the column
    count = data[feature].nunique()
    if n is None:
        plt.figure(figsize=(count + 2, 6))
    else:
        plt.figure(figsize=(n + 2, 6))

    plt.xticks(rotation=90, fontsize=15)
    ax = sns.countplot(
        data=data,
        x=feature,
        palette="Accent",
        order=data[feature].value_counts().index[:n],
    )

    for p in ax.patches:
        if perc == True:
            label = "{:.1f}%".format(
                100 * p.get_height() / total
            )  # percentage of each class of the category
        else:
            label = p.get_height()  # count of each level of the category

        x = p.get_x() + p.get_width() / 2  # width of the plot
        y = p.get_height()  # height of the plot

        ax.annotate(
            label,
            (x, y),
            ha="center",
            va="center",
            size=12,
            xytext=(0, 5),
            textcoords="offset points",
        )  # annotate the percentage

    plt.show()  # show the plot

### Dirstribution & Box Plots

In [ ]:
# Plotting histogram and boxplot for Product_Weight
histogram_boxplot(dataset, 'Product_Weight')

The distribution of Product_Weight appear relatively uniform, indicating a balanced spread of values across its range.

In [ ]:
# Plotting histogram and boxplot for Product_MRP
histogram_boxplot(dataset, 'Product_MRP')

The distribution of Product_MRP appear relatively uniform, indicating a balanced spread of values across its range.

In [ ]:
# Plotting histogram and boxplot for Product_Store_Sales_Total
histogram_boxplot(dataset, 'Product_Store_Sales_Total')

The distribution of Product_Store_Sales_Total appear relatively uniform, indicating a balanced spread of values across its range.

In [ ]:
# Plotting histogram and boxplot for Product_Allocated_Area
histogram_boxplot(dataset, 'Product_Allocated_Area')

Product_Allocated_Area exhibits a right-skewed distribution, suggesting that most products occupy smaller areas, with a few products allocated significantly larger space.


### Labeled Bar Plots

In [ ]:
# Plotting a labeled bar chart showing Product_Sugar_Content distribution
labeled_barplot(dataset, 'Product_Sugar_Content', perc=True)

A majority of products in the dataset are categorized as low-sugar, indicating a strong representation of healthier food options.

In [ ]:
# Plotting a labeled bar chart showing Product_Type distribution
plt.figure(figsize=(10,12))
labeled_barplot(dataset, 'Product_Type', perc=True)

Among all product categories, fruits and vegetables constitute the largest share, highlighting their dominance in the overall product mix.

In [ ]:
# Plotting a labeled bar chart showing Store_Size distribution
labeled_barplot(dataset, 'Store_Size', perc=True)

Most retail outlets fall under the medium store size category, suggesting a standardized store format across locations.

In [ ]:
# Plotting a labeled bar chart showing Store_Location_City_Type distribution
labeled_barplot(dataset, 'Store_Location_City_Type', perc=True)

 A significant proportion of stores are situated in Tier-2 cities, pointing toward a strong retail presence beyond metropolitan areas.

In [ ]:
# Plotting a labeled bar chart showing Store_Type distribution
labeled_barplot(dataset, 'Store_Type', perc=True)

Supermarket Type-2 emerges as the most prevalent store format, outperforming other store types in representation.

## Bivariate Analysis

### Heatmap

In [ ]:
# Including only numerical columns
num_cols = ['Product_Weight', 'Product_Allocated_Area', 'Product_MRP', 'Store_Establishment_Year', 'Product_Store_Sales_Total']
cols_list = [col for col in dataset.columns if col in num_cols]

# plotting the heatmap

plt.figure(figsize=(12, 7))
sns.heatmap(
    dataset[cols_list].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="viridis"
)
plt.show()

- Product_MRP exhibits the strongest positive correlation with Product_Store_Sales_Total, indicating that higher-priced products tend to generate greater sales revenue.

- Product_Weight also shows a notable positive correlation with total sales, suggesting that heavier products may contribute to higher revenue per transaction.

- A negative correlation is observed between Store_Establishment_Year and Product_Store_Sales_Total, implying that older stores generally achieve higher sales compared to newer ones.

- Additionally, Product_MRP and Product_Weight are positively correlated, hinting that larger or heavier products are often priced higher.

In [ ]:
# Function to plot stacked bar plot
def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 5, 5))
    plt.legend(
        loc="lower left", frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

### Stacked Bar Plots

In [ ]:
# Plotting Stacked Bar Plot(Product_Sugar_Content vs Product_Type)
stacked_barplot(dataset, "Product_Sugar_Content", "Product_Type")

- Low-sugar products dominate the dataset, contributing the highest counts across most food categories such as Fruits & Vegetables, Snack Foods, Frozen Foods, and Dairy.

- Regular sugar products form the second-largest share, particularly visible in Snack Foods, Frozen Foods, and Canned items.

- No-sugar products are highly category-specific—they appear exclusively in Health & Hygiene, Household, and Others, indicating these are largely non-edible or non-sweet consumables.

In [ ]:
# Plotting Stacked Bar Plot(Store_Size vs Store_Type)
stacked_barplot(dataset, "Store_Size", "Store_Type")

- It is observed that small-sized stores are predominantly food marts, whereas large-sized stores are mainly Supermarket Type 1.

- Medium-sized stores are largely Supermarket Type 2, with a smaller proportion operating as departmental stores.

In [ ]:
# Plotting Stacked Bar Plot(Store_Location_City_Type vs Store_Type)
stacked_barplot(dataset, "Store_Location_City_Type", "Store_Type")

- Tier 1 cities host only Departmental Stores, indicating a preference for larger, full-format retail outlets in metro areas.

- Tier 3 cities consist exclusively of Food Marts, reflecting demand for smaller, convenience-focused stores in less urbanized locations.

- Tier 2 cities are dominated by Supermarkets, with Supermarket Type 2 being the most prevalent, followed by Supermarket Type 1.

In [ ]:
# Plotting Stacked Bar Plot(Store_Type vs Product_Type)
stacked_barplot(dataset, "Store_Type","Product_Type")

- Supermarket Type 2 dominates across all product categories, contributing the highest number of items in every major segment such as Snack Foods, Fruits & Vegetables, Dairy, and Frozen Foods. This aligns with its highest overall store count.

- Food Marts stock fewer items overall and carry relatively lower counts across categories, consistent with their smaller store size and convenience-oriented model.

- Departmental Stores maintain a balanced product mix, with moderate representation across food and non-food categories such as Health & Hygiene and Household, reflecting a wide-assortment strategy.

- Supermarket Type 1 shows stronger presence in fresh and staple categories like Fruits & Vegetables, Meat, and Dairy compared to Food Marts, suggesting a focus on core grocery items.

## Multivariate Analysis

In [ ]:
# Plotting boxplot to analyze the distribution of total product sales
fig = plt.figure(figsize=(12,10))
sns.boxplot(data=dataset,y= "Store_Type",x= "Product_Store_Sales_Total",hue="Store_Size",palette='cividis')
plt.show()

- Departmental Stores record the highest median sales, indicating stronger overall sales performance compared to other store types. They also show a wide spread, suggesting high variability in sales across locations.

- Supermarket Type 1 and Supermarket Type 2 exhibit similar sales ranges, but Supermarket Type 1 has a slightly higher median, implying marginally better sales consistency.

- Supermarket Type 2 shows more lower-end outliers, indicating that some stores underperform despite the overall moderate-to-high sales range.

- Food Marts have the lowest median sales and the narrowest interquartile range, reflecting limited sales capacity and more consistent but lower sales volumes—typical of small-sized stores.

In [ ]:
# Plotting boxplot to analyze the distribution of Product_MRP and Product_Sugar_Content with respect to Product_Type
fig = plt.figure(figsize=(16,11))
sns.boxplot(data=dataset,y= "Product_Sugar_Content",x= "Product_MRP",hue="Product_Type",palette='cividis')
plt.show()

- Low-sugar and regular-sugar products show very similar MRP distributions across most product categories, indicating that sugar content alone does not significantly drive pricing.

- Regular-sugar products exhibit slightly higher upper-range MRPs and more high-price outliers in categories like Snack Foods, Soft Drinks, and Frozen Foods, suggesting premium variants within regular products.

- No-sugar products are limited to fewer categories and show a wider spread in MRP, particularly in Health & Hygiene, Household, and Others, reflecting pricing differences driven more by product function than ingredients.


# **Data Preprocessing**

### Outlier Detection

In [ ]:
# Outlier detection using boxplots

# Selecting only numeric columns from the dataset
numeric_columns = dataset.select_dtypes(include=np.number).columns.tolist()

# Creating a large figure to fit multiple subplots
plt.figure(figsize=(15, 12))

# Looping through each numeric column
for i, variable in enumerate(numeric_columns):

    # Creating a subplot grid (4 rows x 4 columns)
    plt.subplot(4, 4, i + 1)

    # Plotting boxplot for the current variable
    sns.boxplot(dataset[variable], whis=1.5, palette="rocket")

    # Adjusting spacing between subplots to avoid overlap
    plt.tight_layout()

    # Setting title as the column name
    plt.title(variable)

# Displaying all boxplots
plt.show()


Most outliers appear business-driven and valid; however, one clear data error is identified where total sales are less than MRP and should be treated as an invalid outlier.

### Outlier Treatment

In [ ]:
# Identifying records where total sales are less than MRP (business-rule outlier)
mask = dataset['Product_Store_Sales_Total'] < dataset['Product_MRP']
mask.sum(), len(dataset)

In [ ]:
# Keeping only records where total sales are at least the product MRP
dataset = dataset[
    dataset['Product_Store_Sales_Total'] >= dataset['Product_MRP']
]

In [ ]:
mask = dataset['Product_Store_Sales_Total'] < dataset['Product_MRP']  # identifying records where total sales are less than the product MRP
mask.sum(), len(dataset)  # counting such records and show total number of rows in the dataset


In [ ]:
 # Creating pipeline to preprocess numerical features by scaling them
numeric_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())  # standardizing numerical features to zero mean and unit variance
])

# Creating pipeline to preprocess categorical features using one-hot encoding
categorical_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # one-hot encoding categorical features and safely handle unseen categories
])



In [ ]:
# Defining the target variable to be predicted (total product sales)
target = 'Product_Store_Sales_Total'

# List of numerical features used for model training
numeric_features = [
    'Product_Weight',
    'Product_Allocated_Area',
    'Product_MRP',
    'Store_Establishment_Year'
]


# List of categorical features used for model training
categorical_features = [
    'Product_Sugar_Content',
    'Product_Type',
    'Store_Size',
    'Store_Location_City_Type',
    'Store_Type'
]


# Combining numerical and categorical preprocessing into a single transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        # applying numeric preprocessing pipeline to numerical columns

        ('cat', categorical_pipeline, categorical_features)
        # applying categorical preprocessing pipeline to categorical columns
    ]
)



In [ ]:
# Selecting input features (both numerical and categorical columns)
X = dataset[numeric_features + categorical_features]

# Selecting target variable to be predicted
y = dataset[target]

In [ ]:
# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# **Model Building**

## Define functions for Model Evaluation

In [ ]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mean_absolute_percentage_error(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

### Metrics of Choice

MAPE is preferred as the metric of choice over RMSE and MAE as it provides an intuitive, normalized measure of prediction accuracy and better supports model comparison and business decision-making.

## Model Building

## Random Forest Regressor

In [ ]:
# Initializing Random Forest regression model with fixed random state
rf_model = RandomForestRegressor(random_state=42)

# Creating a pipeline that first preprocesses data, then trains the model
rf_pipeline = make_pipeline(preprocessor, rf_model)

# Training the pipeline using training features and target values
rf_pipeline.fit(X_train, y_train)

In [ ]:
# Evaluating model performance on training data
train_perf = model_performance_regression(rf_pipeline, X_train, y_train)
train_perf  # Displaying training performance metrics


In [ ]:
# Evaluating model performance on test data
test_perf = model_performance_regression(rf_pipeline,X_test,y_test)
test_perf  # Displaying test performance metrics

- The model performs exceptionally well on training data (very low RMSE/MAE, R² ≈ 0.99, and MAPE ≈ 1.4%), indicating an excellent fit.

- On test data, performance drops (higher RMSE/MAE, R² ≈ 0.93, MAPE ≈ 4.1%), which suggests some level of overfitting, though the model still generalizes reasonably well.

### Let’s apply hyperparameter tuning to reduce overfitting and improve test performance.

## Random Forest Regressor - Hyperparameter Tuning

In [ ]:
# Initializing a new Random Forest regressor for hyperparameter tuning
rf_tuned = RandomForestRegressor(random_state=42)

# Creating a pipeline combining preprocessing and the Random Forest model
rf_pipeline = make_pipeline(preprocessor, rf_tuned)

# Defining the hyperparameter grid for tuning
parameters = {
    'randomforestregressor__max_depth': [3, 4, 5, 6],  # depth of each tree
    'randomforestregressor__max_features': ['sqrt', 'log2', None],  # number of features considered at each split
    'randomforestregressor__n_estimators': [50, 75, 100, 125, 150]  # number of trees in the forest
}

# Defining the scoring metric as R-squared
scorer = metrics.make_scorer(metrics.r2_score)

# Performing grid search with 5-fold cross-validation
grid_obj = GridSearchCV(rf_pipeline, param_grid=parameters, scoring=scorer, cv=5)

# Fitting grid search to training data to find the best hyperparameters
grid_obj = grid_obj.fit(X_train, y_train)

# Retrieving the best model from grid search
rf_tuned = grid_obj.best_estimator_

# Training the tuned Random Forest pipeline on the full training set
rf_tuned.fit(X_train, y_train)


In [ ]:
# Evaluating model performance on training data
train_perf_rfhp = model_performance_regression(rf_tuned,X_train,y_train)
train_perf_rfhp # Displaying training performance metrics

In [ ]:
# Evaluating model performance on test data
test_perf_rfhp = model_performance_regression(rf_tuned,X_test,y_test)
test_perf_rfhp # Displaying test performance metrics

- The model shows similar performance on training and test data, with only a small increase in RMSE, MAE, and MAPE on the test set.

- The close R² values (Train ≈ 0.93, Test ≈ 0.91) indicate reduced overfitting and better generalization compared to earlier results.

- Although overall accuracy is slightly lower, the model is more stable and reliable on unseen data after tuning.

## XGBoost Regressor

In [ ]:
# Initializing XGBoost regressor with a fixed random state
xgbmodel = XGBRegressor(random_state=42)

# Creating a pipeline combining preprocessing and the XGBoost model
xgb_pipeline = make_pipeline(preprocessor, xgbmodel)

# Training the XGBoost pipeline on the training data
xgb_pipeline.fit(X_train, y_train)


In [ ]:
# Evaluating model performance on training data
xgb_train_perf = model_performance_regression(xgb_pipeline, X_train, y_train)
xgb_train_perf # Displaying training performance metrics

In [ ]:
# Evaluating model performance on test data
xgb_test_perf = model_performance_regression(xgb_pipeline, X_test, y_test)
xgb_test_perf # Displaying test performance metrics

- The model demonstrates very strong training performance (R² ≈ 0.99, low RMSE/MAE, and MAPE ≈ 2.1%), indicating an excellent fit on the training data.

- However, test performance drops notably (higher RMSE/MAE, R² ≈ 0.92, MAPE ≈ 5.0%), suggesting the model is still overfitting.


### Let’s apply hyperparameter tuning to reduce overfitting and improve test performance.

## XGBoost Regressor- Hyperparameter Tuning

In [ ]:
# Initializing XGBoost regressor with a fixed random state
xgbmodel = XGBRegressor(random_state=42)

# Creating a pipeline combining preprocessing and the XGBoost model
xgbpipelineht = make_pipeline(preprocessor, xgbmodel)

# Defining hyperparameter grid for tuning XGBoost
param_grid = {
    'xgbregressor__n_estimators': [50, 100, 150, 200],       # number of boosting rounds
    'xgbregressor__max_depth': [2, 3, 4],                     # maximum tree depth
    'xgbregressor__learning_rate': [0.01, 0.05, 0.1],         # step size shrinkage
    'xgbregressor__colsample_bytree': [0.4, 0.5, 0.6],        # subsample ratio of columns per tree
    'xgbregressor__colsample_bylevel': [0.4, 0.5, 0.6],       # subsample ratio per tree level
    'xgbregressor__reg_lambda': [0.4, 0.5, 0.6]              # L2 regularization term
}

# Defining scoring metric as R-squared
scorer = metrics.make_scorer(metrics.r2_score)

# Setting up grid search with 5-fold cross-validation
grid_obj = GridSearchCV(
    xgbpipelineht,
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=1
)

# Fitting grid search to training data to find the best hyperparameters
grid_obj.fit(X_train, y_train)

# Retrieving the best pipeline from grid search
xgb_tuned = grid_obj.best_estimator_

# Training the tuned XGBoost pipeline on the full training set
xgb_tuned.fit(X_train, y_train)


In [ ]:
# Evaluating model performance on training data
xgbtuned_train_performance = model_performance_regression(xgb_tuned, X_train, y_train)
xgbtuned_train_performance # Displaying training performance metrics

In [ ]:
# Evaluating model performance on test data
xgbtuned_test_performance = model_performance_regression(xgb_tuned, X_test, y_test)
xgbtuned_test_performance # Displaying test performance metrics

- The tuned model shows slightly reduced training performance compared to the untuned version, with lower R² and marginally higher RMSE/MAE, indicating that the model complexity has been effectively controlled.
- On the test data, performance improves and becomes more consistent with the training results (R² ≈ 0.92, lower error gap, and MAPE ≈ 5%), demonstrating better generalization.

# **Model Performance Comparison, Final Model Selection, and Serialization**

In [ ]:
# Combining training performance metrics of all models into a single DataFrame for comparison
models_train_comp_df = pd.concat(
    {
        'Random Forest Estimator': train_perf,
        'Random Forest Estimator-Tuned': train_perf_rfhp,
        'XGBoost Estimator': xgb_train_perf,
        'XGBoost Estimator-Tuned': xgbtuned_train_performance,
    },
    axis=1
)

# Transposing the DataFrame to have models as rows and metrics as columns for easier readability
models_train_comp_df.T


In [ ]:
# Combining test performance metrics of all models into a single DataFrame for comparison
models_test_comp_df = pd.concat(
    {
        'Random Forest Estimator': test_perf,
        'Random Forest Estimator-Tuned': test_perf_rfhp,
        'XGBoost Estimator': xgb_test_perf,
        'XGBoost Estimator-Tuned': xgbtuned_test_performance,
    },
    axis=1
)
# Transposing the DataFrame to have models as rows and metrics as columns for easier readability
models_test_comp_df.T

In [ ]:
# Calculating the difference in performance metrics between training and test sets
diff = (models_train_comp_df - models_test_comp_df)

# Transposing the difference DataFrame to have models as rows for easier comparison
diff.T


The tuned Random Forest regressor is selected as the final model as it delivers the lowest prediction error across RMSE, MAE, and MAPE, making it the most reliable and business-ready model despite a marginally lower R² compared to untuned XGBoost.”

## Model Serialization

In [ ]:
# File path where the trained sales prediction model is saved
saved_model_path = '/content/drive/MyDrive/backend_files/product_total_sales_prediction_model_v1_1.joblib'

In [ ]:
# Serializing and saving the model to disk for future inference
joblib.dump(xgb_tuned,saved_model_path)

In [ ]:
# Loading the trained sales prediction model from disk for inference
saved_model = joblib.load(saved_model_path)
saved_model

In [ ]:
# Generating predictions on the test dataset using the loaded trained model
saved_model.predict(X_test)

# **Deployment - Backend**

## Flask Web Framework


In [ ]:
%%writefile /content/drive/MyDrive/backend_files/app.py

import numpy as np
import pandas as pd
import joblib
from flask import Flask, request, jsonify
import os

# Initializing Flask application for Superkart sales prediction API
total_sales_predictor_api = Flask("Superkart Total Sales Predictor")

# Defining base directory and model path
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
MODEL_PATH = os.path.join(BASE_DIR, "product_total_sales_prediction_model_v1_1.joblib")

# Loading the trained and saved sales prediction model
model = joblib.load(MODEL_PATH)

@total_sales_predictor_api.get('/')
def home():
    """
    Health check endpoint to verify API availability
    """
    return "Superkart Total Sales Predictor"

@total_sales_predictor_api.post('/v1/sales')
def predict_total_sales():
    """
    Predict total sales for a single product-store combination
    """

    # Parsing input JSON payload
    product_data = request.get_json()

    # Preparing input features in the expected model format
    sample = {
        'Product_Weight': product_data['Product_Weight'],
        'Product_Sugar_Content': product_data['Product_Sugar_Content'],
        'Product_Allocated_Area': product_data['Product_Allocated_Area'],
        'Product_Type': product_data['Product_Type'],
        'Product_MRP': product_data['Product_MRP'],
        'Store_Establishment_Year': product_data['Store_Establishment_Year'],
        'Store_Size': product_data['Store_Size'],
        'Store_Location_City_Type': product_data['Store_Location_City_Type'],
        'Store_Type': product_data['Store_Type']
    }

    # Converting input dictionary to DataFrame for model inference
    input_data = pd.DataFrame([sample])

    # Generating sales prediction
    predicted_sales = model.predict(input_data)[0]


    # Rounding prediction to two decimal places for readability
    predicted_sales = round(float(predicted_sales), 2)

    # Returning prediction as JSON response
    return jsonify({"predicted_sales": predicted_sales})


@total_sales_predictor_api.post('/v1/salesbatch')
def predict_total_sales_batch():
    """
    Predict total sales for multiple records provided as a CSV file
    """

    # Reading uploaded CSV file
    file = request.files['file']
    input_data = pd.read_csv(file)

    # Generating predictions for batch input
    predicted_total_sales = model.predict(input_data).tolist()

    # Rounding prediction to two decimal places for readability
    predicted_prices = [round(float(x), 2) for x in predicted_total_sales]

    # Extracting product IDs to map predictions
    product_ids = input_data['Product_Id'].tolist()

    # Creating response dictionary mapping ID to predicted sales
    output_dict = dict(zip(product_ids, predicted_prices))

    return output_dict


if __name__ == "__main__":
    # Running Flask application in debug mode (disable in production)
    total_sales_predictor_api.run(debug=True)


## Dependencies File

In [ ]:
%%writefile /content/drive/MyDrive/backend_files/requirements.txt
pandas==2.2.2
numpy==2.0.2
scikit-learn==1.6.1
xgboost==2.1.4
joblib==1.4.2
Werkzeug==2.2.2
flask==2.2.2
gunicorn==20.1.0
requests==2.28.1
uvicorn[standard]
streamlit==1.43.2

## Dockerfile

In [ ]:
%%writefile /content/drive/MyDrive/backend_files/Dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY . .

RUN pip install --no-cache-dir --upgrade -r requirements.txt

CMD ["gunicorn", "-w", "4", "-b", "0.0.0.0:7860", "app:total_sales_predictor_api"]



## Setting up a Hugging Face Docker Space for the Backend

In [ ]:
import getpass

# Prompting to securely enter Hugging Face access token without displaying it on the screen
accesstoken = getpass.getpass("Enter your access token:")

In [ ]:
from huggingface_hub import login

# Logging in to Hugging Face using your account-level access token
# This token allows us to create repositories, upload files, and manage Spaces
login(token=accesstoken)

In [ ]:
# Importing function to programmatically create a new Hugging Face repository or Space
from huggingface_hub import create_repo

try:
    # Creating a new Hugging Face Space for the backend application using Docker SDK
    create_repo(
        "TotalSalesPredictionBackend",  # Name of the Space
        repo_type="space",              # Specifying that this is a Space repository
        space_sdk="docker",             # Using Docker SDK for containerized backend
        private=False                    # Making the Space public
    )
    print("Backend Space created successfully")

except Exception as e:
    # Handling the case where the Space already exists
    if "RepositoryAlreadyExistsError" in str(e):
        print("Repository Already Exists")
    else:
        # Catching and displaying any other errors during Space creation
        print(f"Error Creating Repository: {e}")


## Uploading Files to Hugging Face Space (Docker Space)

In [ ]:
from huggingface_hub import HfApi

# Defining the Hugging Face Space repository ID for the backend
repo_id = "Amidho/TotalSalesPredictionBackend"

# Initializing Hugging Face API client
api = HfApi()

# Uploading all backend files (Flask app, model, etc.) to the Hugging Face Space
api.upload_folder(
    folder_path="/content/drive/MyDrive/backend_files",  # Local folder containing backend files
    repo_id=repo_id,                                     # Target Hugging Face Space repository
    repo_type="space"                                    # Specify repository type as Space
)


## Link of the Hugging Face space(Backend):

https://huggingface.co/spaces/Amidho/TotalSalesPredictionBackend

# **Deployment - Frontend**

## Streamlit for Interactive UI

In [ ]:
%%writefile /content/drive/MyDrive/frontend_files/app.py

import streamlit as st
import pandas as pd
import requests

# App title and description
st.title("Total Sales Price Prediction")
st.subheader("Superkart Online Prediction")


# Numeric input fields

Product_Weight = st.number_input(
    "Product Weight",
    min_value=0.0,
    step=0.1
)

Product_Allocated_Area = st.number_input(
    "Product Allocated Area",
    min_value=0.0,
    step=1.0
)

Product_MRP = st.number_input(
    "Product MRP",
    min_value=0.0,
    step=1.0
)



Store_Establishment_Year = st.number_input(
    "Store Establishment Year",
    min_value=1900,
    max_value=2026,
    step=1
)


# Categorical input fields

Product_Sugar_Content = st.selectbox(
    "Product Sugar Content",
    [
        "Low Sugar",
        "Regular",
        "No Sugar"
    ]
)

Product_Type = st.selectbox(
    "Product Type",
    [
        "Meat",
        "Snack Foods",
        "Hard Drinks",
        "Dairy",
        "Canned",
        "Soft Drinks",
        "Health and Hygiene",
        "Baking Goods",
        "Bread",
        "Breakfast",
        "Frozen Foods",
        "Fruits and Vegetables",
        "Household",
        "Seafood",
        "Starchy Foods",
        "Others"
    ]
)

Store_Size = st.selectbox(
    "Store Size",
    ["Small", "Medium", "High"]
)

Store_Location_City_Type = st.selectbox(
    "City Type",
    ["Tier 1", "Tier 2", "Tier 3"]
)

Store_Type = st.selectbox(
    "Store Type",
    [
        "Supermarket Type1",
        "Supermarket Type2",
        "Departmental Store",
        "Food Mart"
    ]
)


# Preparing input data for API call

input_data = pd.DataFrame([{
    "Product_Weight": Product_Weight,
    "Product_Sugar_Content": Product_Sugar_Content,
    "Product_Allocated_Area": Product_Allocated_Area,
    "Product_Type": Product_Type,
    "Product_MRP": Product_MRP,
    "Store_Establishment_Year": Store_Establishment_Year,
    "Store_Size": Store_Size,
    "Store_Location_City_Type": Store_Location_City_Type,
    "Store_Type": Store_Type
}])


# Single prediction API call

if st.button("Predict"):
    response = requests.post(
        "https://Amidho-TotalSalesPredictionBackend.hf.space/v1/sales",
        json=input_data.to_dict(orient='records')[0]
    )

    if response.status_code == 200:
        prediction = response.json()["predicted_sales"]
        st.success(f"Predicted Total Sales: {prediction}")
    else:
        st.error("Error Making Prediction")


# Batch prediction section

st.subheader("Batch Prediction")

uploaded_file = st.file_uploader(
    "Upload csv file for batch prediction:",
    type=["csv"]
)

if uploaded_file is not None:
    if st.button("Predict Batch"):
        response = requests.post(
            "https://Amidho-TotalSalesPredictionBackend.hf.space/v1/salesbatch",
            files={"file": uploaded_file}
        )

        if response.status_code == 200:
            predictions = response.json()
            st.success("Batch Prediction Successful")
            st.write(predictions)
        else:
            st.error("Error Making Batch Prediction")


## Dependencies File

In [ ]:
%%writefile /content/drive/MyDrive/frontend_files/requirements.txt
pandas==2.2.2
requests==2.28.1
streamlit==1.43.2

## DockerFile

In [ ]:
%%writefile /content/drive/MyDrive/frontend_files/Dockerfile
# Use a minimal base image with Python 3.9 installed
FROM python:3.9-slim

# Set the working directory inside the container to /app
WORKDIR /app

# Copy all files from the current directory on the host to the container's /app directory
COPY . .

# Install Python dependencies listed in requirements.txt
RUN pip3 install -r requirements.txt

# Define the command to run the Streamlit app on port 7860 and make it accessible externally
CMD ["streamlit", "run", "app.py", "--server.port=7860", "--server.address=0.0.0.0", "--server.enableXsrfProtection=false"]

# NOTE: Disable XSRF protection for easier external access in order to make batch predictions

## Uploading Files to Hugging Face Space (Streamlit Space)

In [ ]:
from huggingface_hub import create_repo

try:
    # Creating a new Hugging Face Space for the frontend application using Docker SDK
    create_repo(
        "TotalSalesPredictionFrontend",
        repo_type="space",
        space_sdk="docker",
        private=False
    )

    print("Frontend Space created successfully")

except Exception as e:
    # Handling case where the Space already exists
    if "RepositoryAlreadyExistsError" in str(e):
        print("Repository Already Exists")
    else:
        # Catching and displaying any other errors during Space creation
        print(f"Error Creating Repository: {e}")


In [ ]:
from huggingface_hub import HfApi

# Defining the Hugging Face Space repository ID (username/space-name)
repo_id = "Amidho/TotalSalesPredictionFrontend"

# Initializing Hugging Face API client
api = HfApi()

# Uploading all frontend files to the Hugging Face Space repository
api.upload_folder(
    folder_path="/content/drive/MyDrive/frontend_files",  # Local folder containing Streamlit frontend files
    repo_id=repo_id,                                      # Target Hugging Face Space
    repo_type="space"                                     # Specify repository type as Space
)


## Link of the Hugging Face space(Frontend):

https://huggingface.co/spaces/Amidho/TotalSalesPredictionFrontend

# **Actionable Insights and Business Recommendations**

- The tuned Random Forest model offers the best balance between accuracy, robustness, and scalability for enterprise use.

- SuperKart should deploy the tuned Random Forest model as its core sales forecasting solution, as it delivers the lowest prediction errors and robust performance across store formats.

- While overall variance explanation remains modest, the model provides sufficiently accurate forecasts for operational decision-making. By combining this model with regional adjustments, confidence intervals, and enriched feature inputs, SuperKart can significantly improve inventory efficiency, reduce stock risks, and enable data-driven sales strategies across its retail network.